In [70]:
import pandas as pd
from ftplib import FTP
from datetime import datetime
import time
import numpy as np
import pymongo
from pymongo import MongoClient

from geopy import distance

import get_recent_days as gtdys
import muni_etl
import labelling as lblng

In [145]:
client = MongoClient('localhost', 27017)

db = client['testavl']
in_coll = db['clean']
out_coll = db['clean_out']

In [149]:
labeler = lblng.Labeling(in_coll, out_coll)

In [153]:
labeler.label_single_starts()

96
58
0
1
207
66
9
